In [ ]:
# 1. Giải bài toán 8 ô chữ bằng A* theo tree search sử dụng SimpleAI với 2 heuristic H1 và H2
# Mục tiêu:

# Cài đặt bài toán 8 ô chữ (ví dụ: ghép chữ thành từ mục tiêu)
# Định nghĩa 2 heuristic H1, H2
# So sánh thời gian và chi phí (số bước) giữa H1 và H2

# 1. Giải bài toán 8 ô chữ bằng A* theo Tree Search với 2 heuristic H1 và H2
from simpleai.search import SearchProblem, astar
import time

GOAL = 'ABCDEFGH'  # Mục tiêu

# Problem với heuristic H1
class CrosswordProblemH1(SearchProblem):
    def actions(self, state):
        # Trả về tất cả các cặp hoán vị
        return [(i, j) for i in range(len(state)) for j in range(i+1, len(state))]

    def result(self, state, action):
        i, j = action
        lst = list(state)
        lst[i], lst[j] = lst[j], lst[i]
        return ''.join(lst)

    def is_goal(self, state):
        return state == GOAL

    def heuristic(self, state):
        # H1: số chữ cái sai vị trí
        return sum(1 for i in range(len(GOAL)) if state[i] != GOAL[i])

# Problem với heuristic H2
class CrosswordProblemH2(SearchProblem):
    def actions(self, state):
        return [(i, j) for i in range(len(state)) for j in range(i+1, len(state))]

    def result(self, state, action):
        i, j = action
        lst = list(state)
        lst[i], lst[j] = lst[j], lst[i]
        return ''.join(lst)

    def is_goal(self, state):
        return state == GOAL

    def heuristic(self, state):
        # H2: số chữ cái sai vị trí + số chữ cái không có trong goal
        misplaced = sum(1 for i in range(len(GOAL)) if state[i] != GOAL[i])
        not_in_goal = len(set(GOAL) - set(state))
        return misplaced + not_in_goal

# --- Chạy A* với H1 ---
start = time.time()
problem_h1 = CrosswordProblemH1('HGFEDCBA')
result_h1 = astar(problem_h1, graph_search=False)  # Tree search
time_h1 = time.time() - start

# --- Chạy A* với H2 ---
start = time.time()
problem_h2 = CrosswordProblemH2('HGFEDCBA')
result_h2 = astar(problem_h2, graph_search=False)  # Tree search
time_h2 = time.time() - start

# --- In kết quả ---
print("H1 path (các hành động):", result_h1.path())
print("H1 số bước:", len(result_h1.path()))
print("H1 thời gian:", time_h1)

print("H2 path (các hành động):", result_h2.path())
print("H2 số bước:", len(result_h2.path()))
print("H2 thời gian:", time_h2)

# H2 thường tốt hơn H1 về thời gian và chi phí vì heuristic đánh giá chính xác hơn.

In [ ]:
# 2. Giải bài toán xếp hậu 8x8 bằng Genetic Algorithm (GA) với SimpleAI
# Yêu cầu:

# Hàm fitness đánh giá số cặp hậu không tấn công nhau (càng cao càng tốt)
# Sinh ngẫu nhiên cá thể
# Crossover và mutate
# Mutation chance < 5% (0.05)

from simpleai.search import SearchProblem
from simpleai.search.local import genetic
import random

N = 8

class NQueensProblem(SearchProblem):
    def generate_random_state(self):
        # Mỗi phần tử là vị trí hàng của hậu ở cột i
        return tuple(random.randint(0, N-1) for _ in range(N))

    def crossover(self, state1, state2):
        cut = random.randint(1, N-1)
        return state1[:cut] + state2[cut:]

    def mutate(self, state):
        index = random.randint(0, N-1)
        new_row = random.randint(0, N-1)
        state_list = list(state)
        state_list[index] = new_row
        return tuple(state_list)

    def value(self, state):
        # Tính số cặp hậu không tấn công nhau
        non_attacking = 0
        for i in range(N):
            for j in range(i+1, N):
                if state[i] != state[j] and abs(state[i] - state[j]) != j - i:
                    non_attacking += 1
        return non_attacking

problem = NQueensProblem()

result = genetic(problem, population_size=100, mutation_chance=0.05, iterations_limit=1000)

print("Giải pháp tốt nhất:", result.state)
print("Fitness:", problem.value(result.state))


In [ ]:
# 3. Dùng SimpleAI chạy Simulated Annealing (SA) và Hill Climbing (HC) 
# ngẫu nhiên 10 lần cho bài toán xếp hậu 8x8, lấy kết quả tốt nhất

from simpleai.search import SearchProblem
from simpleai.search.local import simulated_annealing, hill_climbing
import random

N = 8  # Số quân hậu

# --- Định nghĩa Problem ---
class NQueensProblem(SearchProblem):
    def __init__(self):
        # Trạng thái ban đầu: random column cho từng hàng
        initial_state = [random.randint(0, N-1) for _ in range(N)]
        super().__init__(initial_state)

    def actions(self, state):
        actions = []
        for row in range(N):
            for col in range(N):
                if col != state[row]:
                    actions.append((row, col))
        return actions

    def result(self, state, action):
        row, col = action
        new_state = list(state)
        new_state[row] = col
        return new_state

    def value(self, state):
        # Fitness = số cặp quân hậu không tấn công nhau
        non_attacking = 0
        for i in range(N):
            for j in range(i+1, N):
                if state[i] != state[j] and abs(state[i]-state[j]) != j-i:
                    non_attacking += 1
        return non_attacking

# --- Hàm hiển thị bàn cờ ---
def print_board(state):
    for row in range(N):
        line = ['Q' if state[row]==col else '.' for col in range(N)]
        print(' '.join(line))
    print()

# --- Chạy SA và HC ---
best_sa = None
best_hc = None
best_sa_value = -1
best_hc_value = -1

for i in range(10):
    problem = NQueensProblem()
    
    sa_result = simulated_annealing(problem, iterations_limit=1000)
    hc_result = hill_climbing(problem, iterations_limit=1000)

    sa_val = problem.value(sa_result.state)
    hc_val = problem.value(hc_result.state)

    if sa_val > best_sa_value:
        best_sa_value = sa_val
        best_sa = sa_result.state

    if hc_val > best_hc_value:
        best_hc_value = hc_val
        best_hc = hc_result.state

# --- In kết quả tốt nhất ---
print("=== Kết quả tốt nhất Simulated Annealing ===")
print_board(best_sa)
print("Fitness:", best_sa_value)

print("=== Kết quả tốt nhất Hill Climbing ===")
print_board(best_hc)
print("Fitness:", best_hc_value)



In [ ]:
# import đúng
from easyAI.TwoPlayerGame import TwoPlayerGame
from easyAI import Human_Player, AI_Player, Negamax

# ANSI color codes
RED = '\033[91m'
GREEN = '\033[92m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
RESET = '\033[0m'

class GameController(TwoPlayerGame):  # <--- sửa ở đây
    def __init__(self, players):
        self.players = players
        self.current_player = 1  # player 1 bắt đầu
        self.board = [0] * 9
    
    def possible_moves(self):
        return [i + 1 for i, b in enumerate(self.board) if b == 0]
    
    def make_move(self, move):
        self.board[int(move) - 1] = self.nplayer
    
    def loss_condition(self):
        combos = [
            [1,2,3],[4,5,6],[7,8,9],
            [1,4,7],[2,5,8],[3,6,9],
            [1,5,9],[3,5,7]
        ]
        opponent = 3 - self.current_player
        return any(all(self.board[i-1] == opponent for i in combo) for combo in combos)
    
    def is_over(self):
        return len(self.possible_moves())==0 or self.loss_condition()
    
    def show(self):
        print(f"\n{BLUE}=== TIC-TAC-TOE ==={RESET}")
        print(f"{YELLOW}Vị trí bảng:{RESET}")
        print("1 2 3\n4 5 6\n7 8 9")
        print(f"{YELLOW}Trạng thái hiện tại:{RESET}")
        
        color_symbols = []
        for i,v in enumerate(self.board):
            if v==0: color_symbols.append(str(i+1))
            elif v==1: color_symbols.append(GREEN+'O'+RESET)
            else: color_symbols.append(RED+'X'+RESET)
        print('\n' + '\n'.join([' '.join(color_symbols[3*j:3*j+3]) for j in range(3)]))
        print()
    
    def scoring(self):
        return -100 if self.loss_condition() else 0
    
    def announce_winner(self):
        print(f"\n{BLUE}=== KẾT QUẢ GAME ==={RESET}")
        if self.loss_condition():
            winner = 3 - self.current_player
            if winner==1:
                print(f"{GREEN}🎉 Player 1 (O) THẮNG! 🎉{RESET}")
            else:
                print(f"{RED}🎉 Player 2 (X) THẮNG! 🎉{RESET}")
        else:
            print(f"{YELLOW}🤝 GAME HÒA! 🤝{RESET}")

def main():
    print(f"{BLUE}=== WELCOME TO TIC-TAC-TOE ==={RESET}")
    print(f"{GREEN}Bạn là Player 1 (O){RESET}")
    print(f"{RED}AI là Player 2 (X){RESET}")
    print("Nhập số từ 1-9 để chọn vị trí đánh")
    print("="*35)
    
    algorithm = Negamax(7)
    game = GameController([Human_Player(), AI_Player(algorithm)])
    game.play()
    game.announce_winner()

if __name__=="__main__":
    main()
